In [3]:
import pandas as pd 
from random import gauss as gs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

import itertools
#from pmdarima import auto_arima

#statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX

%matplotlib inline

In [ ]:
# average(median) price across time with Texas Zipcode (most expensive, least expensive)
# pull out top 10 least expensive zipcode 
# train 15 years of data 
# rmse/mse 
# baseline -> shift 
# from sklearn.model_selection import TimeSeriesSplit

In [4]:
df = pd.read_csv("time-series/zillow_data.csv")

In [91]:
df
#df[(df['State'] == 'IL') & (df['RegionName'] > 60020)].sort_values('RegionName')


,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,58333,1338,Ashfield,MA,Greenfield Town,Franklin,14719,94600.0,94300.0,94000.0,...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,59107,3293,Woodstock,NH,Claremont,Grafton,14720,92700.0,92500.0,92400.0,...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,75672,40404,Berea,KY,Richmond,Madison,14721,57100.0,57300.0,57500.0,...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,191100.0,192400.0,193700.0,...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [92]:
df.describe()

,RegionID,RegionName,SizeRank,1996-04,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
count,14723.000000,14723.000000,14723.000000,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,1.368400e+04,...,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04,1.472300e+04
mean,81075.010052,48222.348706,7362.000000,1.182991e+05,1.184190e+05,1.185374e+05,1.186531e+05,1.187803e+05,1.189275e+05,1.191205e+05,...,2.733354e+05,2.748658e+05,2.764646e+05,2.780332e+05,2.795209e+05,2.810953e+05,2.826571e+05,2.843687e+05,2.865114e+05,2.880399e+05
std,31934.118525,29359.325439,4250.308342,8.600251e+04,8.615567e+04,8.630923e+04,8.646795e+04,8.665094e+04,8.687208e+04,8.715185e+04,...,3.603984e+05,3.614678e+05,3.627563e+05,3.644610e+05,3.656003e+05,3.670454e+05,3.695727e+05,3.717739e+05,3.724612e+05,3.720544e+05
min,58196.000000,1001.000000,1.000000,1.130000e+04,1.150000e+04,1.160000e+04,1.180000e+04,1.180000e+04,1.200000e+04,1.210000e+04,...,1.440000e+04,1.450000e+04,1.470000e+04,1.480000e+04,1.450000e+04,1.430000e+04,1.410000e+04,1.390000e+04,1.380000e+04,1.380000e+04
25%,67174.500000,22101.500000,3681.500000,6.880000e+04,6.890000e+04,6.910000e+04,6.920000e+04,6.937500e+04,6.950000e+04,6.960000e+04,...,1.269000e+05,1.275000e+05,1.282000e+05,1.287000e+05,1.292500e+05,1.299000e+05,1.306000e+05,1.310500e+05,1.319500e+05,1.324000e+05
50%,78007.000000,46106.000000,7362.000000,9.950000e+04,9.950000e+04,9.970000e+04,9.970000e+04,9.980000e+04,9.990000e+04,9.995000e+04,...,1.884000e+05,1.896000e+05,1.905000e+05,1.914000e+05,1.925000e+05,1.934000e+05,1.941000e+05,1.950000e+05,1.967000e+05,1.981000e+05
75%,90920.500000,75205.500000,11042.500000,1.432000e+05,1.433000e+05,1.432250e+05,1.432250e+05,1.435000e+05,1.437000e+05,1.439000e+05,...,3.050000e+05,3.066500e+05,3.085000e+05,3.098000e+05,3.117000e+05,3.134000e+05,3.151000e+05,3.168500e+05,3.188500e+05,3.211000e+05
max,753844.000000,99901.000000,14723.000000,3.676700e+06,3.704200e+06,3.729600e+06,3.754600e+06,3.781800e+06,3.813500e+06,3.849600e+06,...,1.888990e+07,1.870350e+07,1.860530e+07,1.856940e+07,1.842880e+07,1.830710e+07,1.836590e+07,1.853040e+07,1.833770e+07,1.789490e+07


In [89]:
df_texas = df.State[df.State == 'TX']
df_texas.head()
# texas 

1    TX
2    TX
4    TX
5    TX
8    TX
Name: State, dtype: object

In [6]:
df1 = pd.read_csv("time-series/zillow_homevalue.csv")
df1.sort_values('RegionID')

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31
0,102001,0,United States,Country,NaN,128395.0,128743.0,129108.0,129852.0,130602.0,...,306859.0,311526.0,315661.0,319583.0,323955.0,328698.0,333926.0,339207.0,344773.0,349816.0
684,394297,689,"Aberdeen, SD",Msa,SD,NaN,NaN,NaN,NaN,NaN,...,193979.0,195043.0,195810.0,196315.0,199212.0,202370.0,206895.0,208901.0,211049.0,211969.0
468,394298,468,"Aberdeen, WA",Msa,WA,NaN,NaN,NaN,NaN,NaN,...,284088.0,290095.0,297523.0,300854.0,309856.0,317090.0,325214.0,331120.0,338389.0,344201.0
245,394299,245,"Abilene, TX",Msa,TX,NaN,NaN,NaN,NaN,NaN,...,161423.0,163809.0,165961.0,168580.0,170714.0,172478.0,174349.0,176524.0,179066.0,181340.0
722,394300,730,"Ada, OK",Msa,OK,54981.0,55172.0,55247.0,55430.0,55632.0,...,116982.0,118292.0,119425.0,120529.0,123413.0,126190.0,128983.0,129888.0,131048.0,132205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898,753925,923,"Vineyard Haven, MA",Msa,MA,335574.0,337487.0,338977.0,343163.0,347738.0,...,1003841.0,1017184.0,1028548.0,1037512.0,1056757.0,1076749.0,1086794.0,1085037.0,1081474.0,1087717.0
827,753926,843,"Weatherford, OK",Msa,OK,77171.0,77285.0,77341.0,77525.0,77641.0,...,128941.0,130102.0,131487.0,132247.0,134999.0,137433.0,140078.0,140815.0,141492.0,141827.0
899,753927,924,"Winnemucca, NV",Msa,NV,NaN,NaN,NaN,NaN,NaN,...,275407.0,277665.0,279165.0,280248.0,286357.0,292263.0,300544.0,304189.0,307973.0,309141.0
462,753928,462,"Wisconsin Rapids-Marshfield, WI",Msa,WI,NaN,NaN,NaN,NaN,NaN,...,152896.0,156561.0,158147.0,158602.0,160159.0,161197.0,162276.0,161764.0,162959.0,163711.0


In [65]:
df2 = pd.read_csv("time-series/zillow_forecast.csv")
df2.sort_values('RegionID')

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,BaseDate,2022-06-30,2022-08-31,2023-05-31
6508,58196,6617,1001,Zip,MA,MA,Agawam,Springfield,Hampden County,2022-05-31,0.8,2.7,6.2
4513,58197,4562,1002,Zip,MA,MA,Amherst,Springfield,Hampshire County,2022-05-31,1.6,3.7,6.4
12474,58200,13922,1005,Zip,MA,MA,Barre,Worcester,Worcester County,2022-05-31,1.4,3.7,8.9
7470,58201,7673,1007,Zip,MA,MA,Belchertown,Springfield,Hampshire County,2022-05-31,1.3,3.1,6.0
17588,58202,22238,1008,Zip,MA,MA,Blandford,Springfield,Hampden County,2022-05-31,1.3,3.0,7.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4472,677175,4522,84009,Zip,UT,UT,South Jordan,Salt Lake City,Salt Lake County,2022-05-31,1.8,4.8,11.9
6379,698721,6485,86005,Zip,AZ,AZ,Flagstaff,Flagstaff,Coconino County,2022-05-31,1.9,4.4,11.0
4351,738092,4398,97003,Zip,OR,OR,Beaverton,Portland-Vancouver-Hillsboro,Washington County,2022-05-31,1.2,3.6,7.8
2827,753843,2850,97703,Zip,OR,OR,Bend,Bend-Redmond,Deschutes County,2022-05-31,2.0,4.8,12.0


In [8]:
df2[df2['RegionID'] == 753929]

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,BaseDate,2022-06-30,2022-08-31,2023-05-31


In [9]:
df3 = pd.read_csv("time-series/zillow_rent.csv")
df3.sort_values('RegionID')

,RegionID,RegionName,SizeRank,MsaName,2014-01,2014-02,2014-03,2014-04,2014-05,2014-06,...,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05
1200,58443,1604,2306,"Worcester, MA",1125.0,1123.0,1132.0,1125.0,1137.0,1137.0,...,1766.0,1815.0,1808.0,1795.0,1737.0,1782.0,1821.0,1823.0,1854.0,1913.0
1241,58459,1702,2434,"Boston, MA",1453.0,NaN,NaN,1523.0,1521.0,1542.0,...,1920.0,2037.0,2045.0,2021.0,2104.0,2112.0,2077.0,2111.0,2095.0,2138.0
808,58477,1752,1400,"Boston, MA",1328.0,1356.0,1392.0,1380.0,1386.0,1396.0,...,2014.0,2025.0,2055.0,2013.0,2060.0,2082.0,2098.0,2140.0,2151.0,2200.0
1395,58495,1810,2852,"Boston, MA",1781.0,1592.0,NaN,1706.0,1719.0,1667.0,...,2306.0,2218.0,2305.0,2335.0,2366.0,2288.0,2325.0,2348.0,2421.0,2465.0
1840,58517,1851,4473,"Boston, MA",NaN,1024.0,1024.0,1056.0,1118.0,1154.0,...,1819.0,1862.0,1796.0,1760.0,1732.0,1785.0,1781.0,1822.0,1842.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,417442,85138,2072,"Phoenix, AZ",847.0,840.0,863.0,868.0,886.0,919.0,...,1852.0,1921.0,1986.0,1942.0,1991.0,2055.0,2092.0,2137.0,2044.0,2077.0
2224,417443,85139,7143,"Phoenix, AZ",942.0,940.0,942.0,934.0,943.0,946.0,...,1994.0,1893.0,2029.0,2039.0,1956.0,1974.0,2050.0,2026.0,2028.0,2108.0
221,417444,85142,305,"Phoenix, AZ",941.0,1050.0,1018.0,1021.0,1042.0,1067.0,...,2122.0,2120.0,2125.0,2122.0,2148.0,2156.0,2186.0,2240.0,2255.0,2246.0
1385,422746,75033,2830,"Dallas-Fort Worth, TX",1375.0,1405.0,1364.0,1375.0,1397.0,1426.0,...,1859.0,1897.0,1928.0,1925.0,1961.0,1959.0,1964.0,1996.0,1999.0,2021.0


In [47]:
ex = df[df['RegionID'] == 753844]
#ex['2022-05']
ex['2018-04']


3187    188300
Name: 2018-04, dtype: int64

In [23]:
df3.isnull().sum(axis = 0)

RegionID        0
RegionName      0
SizeRank        0
MsaName         0
2014-01       337
             ... 
2022-01       100
2022-02        68
2022-03        46
2022-04        39
2022-05        49
Length: 105, dtype: int64

In [48]:
date = pd.date_range(start='2014-01', end = '2022-05', freq='M').strftime('%Y-%m')
date
#time_series = pd.Series(ex[date], index=date)

Index(['2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12',
       '2015-01', '2015-02', '2015-03', '2015-04', '2015-05', '2015-06',
       '2015-07', '2015-08', '2015-09', '2015-10', '2015-11', '2015-12',
       '2016-01', '2016-02', '2016-03', '2016-04', '2016-05', '2016-06',
       '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12',
       '2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
       '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
       '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', 

In [68]:
f_fixed = df2.copy()
#f_fixed['profit'] = (f_fixed['2023-05-31'] - f_fixed['2022-06-30'])/f_fixed['2022-06-30']
f_fixed['growth'] = f_fixed['2023-05-31'] - f_fixed['2022-06-30']
f_fixed.sort_values('growth', ascending=False)
#f_fixed


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,BaseDate,2022-06-30,2022-08-31,2023-05-31,growth
18319,69640,23752,27968,Zip,NC,NC,Rodanthe,Kill Devil Hills,Dare County,2022-05-31,3.3,12.9,33.3,30.0
18765,65410,24682,18254,Zip,PA,PA,Tresckow,Allentown-Bethlehem-Easton,Carbon County,2022-05-31,1.3,8.5,28.8,27.5
261,90769,263,75217,Zip,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,2022-05-31,2.9,11.3,28.9,26.0
21121,71051,34302,30332,Zip,GA,GA,Atlanta,Atlanta-Sandy Springs-Roswell,Fulton County,2022-05-31,-1.0,-0.5,23.5,24.5
20770,98303,30304,95607,Zip,CA,CA,Capay,Sacramento--Roseville--Arden-Arcade,Yolo County,2022-05-31,-3.5,-1.1,19.7,23.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21106,67551,32844,23125,Zip,VA,VA,Port Haywood,Virginia Beach-Norfolk-Newport News,Mathews County,2022-05-31,NaN,NaN,-0.1,NaN
21108,77339,32966,44825,Zip,OH,OH,Chatfield,Bucyrus,Crawford County,2022-05-31,NaN,NaN,2.2,NaN
21109,93398,33114,80448,Zip,CO,CO,Grant,Denver-Aurora-Lakewood,Park County,2022-05-31,NaN,0.7,4.9,NaN
21111,93378,33238,80425,Zip,CO,CO,Pine,Denver-Aurora-Lakewood,Jefferson County,2022-05-31,NaN,NaN,1.4,NaN


In [84]:
#f_fixed.drop(columns = f_fixed[['2022-06-30', '2022-08-31', '2023-05-31']].columns[(f_fixed[['2022-06-30', '2022-08-31', '2023-05-31']] > 0).any()])

indexNames = f_fixed[ f_fixed[['2022-06-30', '2022-08-31', '2023-05-31']] > 0 ].index
f_fixed.drop(indexNames , inplace=True)

f_fixed.sort_values(['2022-06-30', '2022-08-31', '2023-05-31'], ascending=True)



,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,BaseDate,2022-06-30,2022-08-31,2023-05-31,growth
